In [1]:
!git clone https://github.com/WorkInTheDark/FairytaleQA_QAG_System.git
%cd ./FairytaleQA_QAG_System/

Cloning into 'FairytaleQA_QAG_System'...
remote: Enumerating objects: 1900, done.
remote: Counting objects: 100% (1900/1900), done.
remote: Compressing objects: 100% (1049/1049), done.
remote: Total 1900 (delta 312), reused 1891 (delta 306), pack-reused 0
Receiving objects: 100% (1900/1900), 7.18 MiB | 14.56 MiB/s, done.
Resolving deltas: 100% (312/312), done.
/content/FairytaleQA_QAG_System


In [4]:
%%capture
!pip install torch==1.8.0
!pip install urllib3==1.25.4
!pip install allennlp-models==1.2
!pip install spacy
!python -m spacy download en_core_web_sm


In [5]:
!pip install future==0.17.1
!pip install pytorch torchvision cudatoolkit=10.2 -c pytorch
!pip install pytorch-lightning==0.8.5
!pip install -U torchtext==0.8.0

     |████████████████████████████████| 829 kB 4.1 MB/s 
  Created wheel for future: filename=future-0.17.1-py3-none-any.whl size=488740 sha256=a9ffc793659844891a15d51e8c5c85f6e03d9f1b465c5d4041a2103435b28555
  Stored in directory: /root/.cache/pip/wheels/16/4c/84/8a6161d44282ede60ed233d090156c6109a7ab865e49c1c9f6
Successfully built future
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
     |████████████████████████████████| 313 kB 4.4 MB/s 
     |████████████████████████████████| 596 kB 43.7 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 6.9 MB 4.1 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninst

In [6]:
#%%capture
%cd ./transformers
!pip install -e ".[testing]"
!pip install -r ./examples/requirements.txt
%cd ./..


/content/FairytaleQA_QAG_System/transformers
Obtaining file:///content/FairytaleQA_QAG_System/transformers
     |████████████████████████████████| 3.0 MB 4.3 MB/s 
     |████████████████████████████████| 41 kB 723 kB/s 
     |████████████████████████████████| 297 kB 82.2 MB/s 
     |████████████████████████████████| 297 kB 51.5 MB/s 
     |████████████████████████████████| 296 kB 74.4 MB/s 
     |████████████████████████████████| 296 kB 77.1 MB/s 
     |████████████████████████████████| 280 kB 92.9 MB/s 
     |████████████████████████████████| 280 kB 74.6 MB/s 
     |████████████████████████████████| 280 kB 77.7 MB/s 
     |████████████████████████████████| 280 kB 90.5 MB/s 
     |████████████████████████████████| 279 kB 38.1 MB/s 
     |████████████████████████████████| 279 kB 67.4 MB/s 
INFO: pip is looking at multiple versions of execnet to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of apipkg to d

/content/FairytaleQA_QAG_System


In [7]:
print(a)

NameError: ignored

In [4]:
%cd ./FairytaleQA_QAG_System/QAG_Generation_E2E

[Errno 2] No such file or directory: './FairytaleQA_QAG_System/QAG_Generation_E2E'
/content/FairytaleQA_QAG_System/QAG_Generation_E2E


In [5]:

import argparse
import json
import time
import warnings
from logging import getLogger
from pathlib import Path
from typing import Dict, List
import argparse
import glob
import logging
import os
import time
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from lightning_base import BaseTransformer, add_generic_args, generic_train
from transformers import MBartTokenizer, T5ForConditionalGeneration
from transformers.modeling_bart import shift_tokens_right

try:
    from .callbacks import Seq2SeqLoggingCallback, get_checkpoint_callback, get_early_stopping_callback
    from .utils import (
        ROUGE_KEYS,
        LegacySeq2SeqDataset,
        Seq2SeqDataset,
        assert_all_frozen,
        calculate_bleu,
        calculate_rouge,
        flatten_list,
        freeze_params,
        get_git_info,
        label_smoothed_nll_loss,
        lmap,
        pickle_save,
        save_git_info,
        save_json,
        use_task_specific_params,
        parse_numeric_cl_kwargs,
    )
except ImportError:
    from callbacks import Seq2SeqLoggingCallback, get_checkpoint_callback, get_early_stopping_callback
    from utils import (
        ROUGE_KEYS,
        LegacySeq2SeqDataset,
        Seq2SeqDataset,
        assert_all_frozen,
        calculate_bleu,
        calculate_rouge,
        flatten_list,
        freeze_params,
        get_git_info,
        label_smoothed_nll_loss,
        lmap,
        pickle_save,
        save_git_info,
        save_json,
        use_task_specific_params,
        parse_numeric_cl_kwargs,
    )

logger = logging.getLogger(__name__)

DEFAULT_DEVICE = "0" if torch.cuda.is_available() else "cpu"


class SummarizationModule(BaseTransformer):
    mode = "summarization"
    loss_names = ["loss"]
    metric_names = ROUGE_KEYS
    default_val_metric = "rouge2"

    def __init__(self, hparams, **kwargs):
        super().__init__(hparams, num_labels=None, mode=self.mode, **kwargs)
        use_task_specific_params(self.model, self.mode)
#         self.target_lens = {
#             "train": self.hparams.max_target_length,
#             "val": self.hparams.val_max_target_length,
#             "test": self.hparams.test_max_target_length,
#         }
#         self.decoder_start_token_id = None  # default to config
#         if self.model.config.decoder_start_token_id is None and isinstance(self.tokenizer, MBartTokenizer):
#             self.decoder_start_token_id = self.tokenizer.lang_code_to_id[hparams.tgt_lang]
#             self.model.config.decoder_start_token_id = self.decoder_start_token_id
#         self.test_max_target_length = self.hparams.test_max_target_length
   

    def freeze_embeds(self):
        """Freeze token embeddings and positional embeddings for bart, just token embeddings for t5."""
        try:
            freeze_params(self.model.model.shared)
            for d in [self.model.model.encoder, self.model.model.decoder]:
                freeze_params(d.embed_positions)
                freeze_params(d.embed_tokens)
        except AttributeError:
            freeze_params(self.model.shared)
            for d in [self.model.encoder, self.model.decoder]:
                freeze_params(d.embed_tokens)

    def forward(self, input_ids, **kwargs):
        return self.model(input_ids, **kwargs)

    def ids_to_clean_text(self, generated_ids: List[int]):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return lmap(str.strip, gen_text)


    def generate(self, input_ids, attention_mask, **generate_kwargs):
        # pad_token_id = self.tokenizer.pad_token_id
        # source_ids, source_mask, y = SummarizationDataset.trim_seq2seq_batch(batch, pad_token_id)
        generated_ids = self.model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            num_beams=1,
            max_length=20,
            min_length=1,
            repetition_penalty=1.5,
            length_penalty=3,
            early_stopping=True,
            use_cache=False,
            **generate_kwargs
        )
        return generated_ids

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]


def generate_summaries_or_translations(
    examples: List[str],
    model_name: str,
    batch_size: int = 8,
    device: str = DEFAULT_DEVICE,
    fp16=False,
    task="summarization",
    prefix='',
    args=None,
    **generate_kwargs,
) -> Dict:
    # save outputs to a list
    output_list = []


    ## 0421 -- load model outside

    # model_name = str(model_name)

    # if "summarization" in args.task:
    #     model: SummarizationModule = SummarizationModule(args)
    # else:
    #     model: SummarizationModule = TranslationModule(args)
    # model = model.load_from_checkpoint(args.ckpt_path)
    # model.eval()
    
    # if args.device != 'cpu':
    #   model.to('cuda:{}'.format(args.device))

    
    # tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=args.cache_dir)
    # tokenizer.add_special_tokens({'sep_token': '<SEP>'})
    # model.resize_token_embeddings(len(tokenizer))
    
    # print('#############################################')
    # print("# model is loaded from", args.ckpt_path)
    # print('# tokenizer.all_special_tokens =', model.tokenizer.all_special_tokens)
    # print('# tokenizer.all_special_ids =', model.tokenizer.all_special_ids)
    # print('#############################################')
    

    # update config with task specific params
    use_task_specific_params(model, task)
    for examples_chunk in tqdm(list(chunks(examples, batch_size))):
        examples_chunk = [prefix + text for text in examples_chunk]
        
        if device=='cpu':
          batch = model.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest")
        else:
          batch = model.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest").to('cuda:{}'.format(device))
        
        ### ORIGINAL NO CPU/GPU support
        ###batch = model.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest")
        
        if len(batch.input_ids[0]) > 1024:
            
            if device=='cpu':
              end_token = model.tokenizer.encode('</s>', return_tensors='pt')[0][-2:]
            else:
              end_token = model.tokenizer.encode('</s>', return_tensors='pt')[0][-2:].to('cuda:{}'.format(device))
            
            ### ORIGINAL NO CPU/GPU support
            ###end_token = model.tokenizer.encode('</s>', return_tensors='pt')[0][-2:]
            
            input_ids = torch.cat((batch.input_ids[0][:1022], end_token), 0).unsqueeze(0)
            batch.input_ids = input_ids
        # batch.input_ids[0,-1] = 2
        # print(batch.input_ids)
        # print(batch.attention_mask)
        # return
        summaries = model.generate(
            input_ids=batch.input_ids,
            attention_mask=batch.attention_mask,
            **generate_kwargs,
        )
        dec = model.tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        for hypothesis in dec:
            output_list.append(hypothesis)

    return output_list


def generate_summaries_or_translations_QA(
    examples: List[str],
    model_name: str,
    batch_size: int = 8,
    device: str = DEFAULT_DEVICE,
    fp16=False,
    task="summarization",
    prefix='',
    args=None,
    **generate_kwargs,
) -> Dict:
    # save outputs to a list
    output_list = []


    # update config with task specific params
    use_task_specific_params(model_rank, task)
    for examples_chunk in tqdm(list(chunks(examples, batch_size))):
        examples_chunk = [prefix + text for text in examples_chunk]
        
        if device=='cpu':
          batch = model_rank.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest")
        else:
          batch = model_rank.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest").to('cuda:{}'.format(device))
        
        ### ORIGINAL NO CPU/GPU support
        ###batch = model_rank.tokenizer(examples_chunk, return_tensors="pt", truncation=True, padding="longest")
        
        if len(batch.input_ids[0]) > 1024:
            
            if device=='cpu':
              end_token = model_rank.tokenizer.encode('</s>', return_tensors='pt')[0][-2:]
            else:
              end_token = model_rank.tokenizer.encode('</s>', return_tensors='pt')[0][-2:].to('cuda:{}'.format(device))
            
            ### ORIGINAL NO CPU/GPU support
            ###end_token = model_rank.tokenizer.encode('</s>', return_tensors='pt')[0][-2:]
            
            input_ids = torch.cat((batch.input_ids[0][:1022], end_token), 0).unsqueeze(0)
            batch.input_ids = input_ids
        # batch.input_ids[0,-1] = 2
        # print(batch.input_ids)
        # print(batch.attention_mask)
        # return
        summaries = model_rank.generate(
            input_ids=batch.input_ids,
            attention_mask=batch.attention_mask,
            **generate_kwargs,
        )
        dec = model_rank.tokenizer.batch_decode(summaries, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        for hypothesis in dec:
            output_list.append(hypothesis)

    return output_list

In [6]:
### we write parser manually because we are not running a file with parameters
### parsers are passed into generate functions 

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", type=str, help="like facebook/bart-large-cnn,t5-base, etc.")
parser.add_argument("--input_path", type=str, help="like cnn_dm/test.source")
parser.add_argument("--save_path", type=str, help="where to save summaries")
parser.add_argument("--reference_path", type=str, required=False, help="like cnn_dm/test.target")
parser.add_argument("--score_path", type=str, required=False, default="metrics.json", help="where to save metrics")
parser.add_argument("--device", type=str, required=False, default=DEFAULT_DEVICE, help="cuda, cuda:1, cpu etc.")
parser.add_argument(
    "--prefix", type=str, required=False, default='', help="will be added to the begininng of src examples"
)
parser.add_argument("--task", type=str, default="summarization", help="used for task_specific_params + metrics")
parser.add_argument("--bs", type=int, default=1, required=False, help="batch size")
parser.add_argument(
    "--n_obs", type=int, default=-1, required=False, help="How many observations. Defaults to all."
)
parser.add_argument("--fp16", action="store_true")
################################
parser.add_argument(
        "--cache_dir",
        type=str,
        help="Where do you want to store the pre-trained models downloaded from s3",
    )
################################
parser.add_argument(
        "--ckpt_path",
        # default="./QG_model_epoch=2_new.ckpt",
        type=str,
        help='path tooo stored model checkpoints',
    )
################################
parser.add_argument("--output_dir", default="./", type=str)
parser.add_argument("--model_name_or_path", type=str, default="facebook/bart-large", help="like facebook/bart-large-cnn,t5-base, etc.")
parser.add_argument("--config_name", type=str, default="facebook/bart-large")
parser.add_argument("--tokenizer_name", type=str, default="facebook/bart-large")
parser.add_argument("--test_max_target_length", type=int)
parser.add_argument("--eval_max_length", type=int)
parser.add_argument("--type_embedding_enabled", type=bool, default=True)
################################
# Unspecified args like --num_beams=2 --decoder_start_token_id=4 are passed to model.generate
################################
args, rest = parser.parse_known_args()

In [7]:
import torch
from allennlp.predictors.predictor import Predictor
import spacy
from spacy import displacy

if DEFAULT_DEVICE != 'cpu':
  predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.11.19.tar.gz", cuda_device=0)
else:
  predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bert-base-srl-2020.11.19.tar.gz", )

nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


downloading: 100%|##########| 405968345/405968345 [00:09<00:00, 43934032.93B/s]


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [8]:
import torch
from allennlp.predictors.predictor import Predictor
import spacy
from spacy import displacy

In [9]:


model: SummarizationModule = SummarizationModule(args)
model_rank: SummarizationModule = SummarizationModule(args)

model = model.load_from_checkpoint("PATH_TO_MODEL/BART_QG/pretrained_BART_on_Fairytale_5e-6_b1_epoch=3.ckpt")

model.eval()

print("finish loading ACQ model")


if DEFAULT_DEVICE != 'cpu':
  model.to('cuda:{}'.format(DEFAULT_DEVICE))
  model_rank.to('cuda:{}'.format(DEFAULT_DEVICE))


Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['encoder.embed_events.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['encoder.embed_events.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

Some weights of BartForConditionalGeneration were not initialized from the model checkpoint at facebook/bart-large and are newly initialized: ['encoder.embed_events.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


finish loading ACQ model


In [10]:
global nlp
global predictor

In [11]:
def run_generate(examples):
  parsed = {}

  generate_results = generate_summaries_or_translations(
      examples,
      args.model_name_or_path,
      batch_size=args.bs,
      device=args.device,
      fp16=args.fp16,
      task=args.task,
      prefix=args.prefix,
      args=args,
      **parsed,
  )

  return generate_results
  

In [12]:
def e2e(input_content, input_category):

  # ==== ANSWER EXTRACTION PROCESS ====
    # extracted results are stored in a list, where each line is an A-C pair
    input_for_ACQ_model = []
    
    doc = nlp(input_content)
    json_samples = []
    for sent in doc.sents:
        json_samples.append({'sentence': sent.text})
    srl_results = predictor.predict_batch_json(json_samples)

    each_answer = ""

    unique_answers = []

    for ent in doc.ents:
        each_answer = ent.text.replace("\n", "")

        each_answer = each_answer.strip('.,\'').strip() + ' .'

        if each_answer in unique_answers:
          continue

        if len(each_answer)>1:
            input_for_ACQ_model.append(each_answer + " </s> " + input_content.replace("\n", ""))
            unique_answers.append( each_answer )
        
    
    for chunk in doc.noun_chunks:
        each_answer = chunk.text.replace("\n", "")

        each_answer = each_answer.strip('.,\'').strip() + ' .'

        if each_answer in unique_answers:
          continue

        if len(chunk.text.split(" ")) >= 2:
            input_for_ACQ_model.append(each_answer + " </s> " + input_content.replace("\n", ""))
            unique_answers.append( each_answer )

    for k in srl_results:
        if k['verbs'] == []:
            continue
        relevant_words = []
        for idx in range(len(k['verbs'][0]['tags'])):
            if k['verbs'][0]['tags'][idx] != 'O':
                relevant_words.append(k['words'][idx])
        target = ' '.join(relevant_words)

        each_answer = target.replace("\n", "").strip('.,\'').strip() + ' .'
        
        if each_answer in unique_answers:
          continue
        
        input_for_ACQ_model.append(each_answer + " </s> " + input_content.replace("\n", ""))
        unique_answers.append( each_answer )

    
    # input_for_ACQ_model = list(set( input_for_ACQ_model ))
    # ==== END OF ANSWER EXTRACTION PROCESS ====



    # ==== QUESTION GENERATION PROCESS ====

    output_for_ACQ_model = run_generate(input_for_ACQ_model)
    # ==== END OF QUESTION GENERATION PROCESS ====


    # ==== POST-PROCESS ====

    for i in range(len(output_for_ACQ_model)):
      output_for_ACQ_model[i] = output_for_ACQ_model[i].replace("\n", "").split("?")[0].strip(' ?') + " ?"

    data_AC_Q = []

    for i in range(len(input_for_ACQ_model)):
        #data_AC_Q.append(input_for_ACQ_model[i].strip().lower() + " </s> " + output_for_ACQ_model[i].strip().lower())
        data_AC_Q.append( [input_for_ACQ_model[i].strip().lower().split(' </s> ')[0], input_for_ACQ_model[i].strip().lower().split(' </s> ')[1], output_for_ACQ_model[i].strip().lower() ] )
    

    return data_AC_Q

In [16]:
!mkdir ./data/generated_qa
!mkdir ./data/generated_qa/val/
!mkdir ./data/generated_qa/test/

mkdir: cannot create directory ‘./data/generated_qa’: File exists
mkdir: cannot create directory ‘./data/generated_qa/val/’: File exists


Here's an example of generate QA-pairs for val split, you may modify the following cell for your own purposes. 

In [ ]:
if __name__ == "__main__":

  book_directory = glob.glob('./data/input_for_QAG/val/*.txt', recursive=True)

  save_directory = glob.glob('./data/generated_qa/val/*.txt', recursive=True)
  saved_list = []
  for i in save_directory:
    book_name = i.split('/')[-1].replace('.txt', '').replace('generated_qa_stories-', '').lower()
    saved_list.append(book_name)


  cnt_book = 0

  for each_book in book_directory:
    book_name = each_book.split('/')[-1].replace('.txt', '').replace('QA_generation_stories-', '').lower()

    if book_name in saved_list:
      continue

    ACQ_pair_list = []
    lines = open(each_book, 'r').readlines()
    for i in lines:
      ACQ_pair_list += e2e(i.replace('\n',''), 'key')
      

    f_out = open('./data/generated_qa/val/generated_qa_stories-' + book_name + ".txt", 'w')

      
    for i in ACQ_pair_list:
      f_out.write(i[0] + ' </s> ' + i[1] + ' </s> ' + i[2] + '\n')
    f_out.close()

    print(cnt_book, book_name)

    cnt_book += 1
    
  
print("book count: ", cnt_book)
  